In [ ]:
# !pip install contractions 

# !{sys.executable} -m pip install contractions

# import sys, setuptools, tokenize
# !pip install pycontractions

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> imports >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

import numpy as np
import pandas as pd
import nltk
import re, unicodedata, string  
# import num2word
# import inflect

from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer

!pip install contractions 
# import sys
import contractions
# import pycontractions 
# from contractions import CONTRACTION_MAP
# from contractions import CONTRACTION_dict
# from pycontractions import Contractions

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> loading the dataset >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# filename = '../dataset/Reviews only (Sentiment).csv'
# file = open(filename, 'rt')
# text = file.read()
# file.close()

df = pd.read_csv('../dataset/testing.csv')
# df_no_indices = df.to_string(index=False)
# print(df_no_indices)
df.head(10)

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing null values >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# counting rows before removing null values
df.shape


In [ ]:
#removing null values
# counting rows after removing null values
df.dropna(axis=0, how="any", thresh=None, subset=None, inplace=False).shape

In [ ]:
# after removing null values
# before removing URL
df.head(10)

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing URL >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_URL(reviewText):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', str(reviewText))

df['reviewText'] = df['reviewText'].apply(remove_URL)

df.head(10)

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing HTML tags >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
def remove_html(reviewText):
    html=re.compile(r'<.*?>')
    return html.sub(r'',str(reviewText))
df['reviewText'] = df['reviewText'].apply(remove_html)

df.head(10)

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing square brackets and the inside of the square bracket >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_between_square_brackets(reviewText):
    return re.sub('\[[^]]*\]', '', str(reviewText))
df['reviewText'] = df['reviewText'].apply(remove_between_square_brackets)
df.head(10)

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing Pictures/Tags/Symbols/Emojis >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def remove_emoji(reviewText):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', str(reviewText))

df['reviewText'] = df['reviewText'].apply(remove_emoji)
df.head(10)

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> expand contractions >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# def expand_contractions(reviewText):
#     contractionsPattern = re.compile('({})'.format('|'.join(CONTRACTION_MAP.keys())),flags=re.IGNORECASE|re.DOTALL)
#     def expand_match(contraction):
#         match = contraction.group(0)
#         firstChar = match[0]
#         expandedContraction = CONTRACTION_MAP.get(match)\
#                         if CONTRACTION_MAP.get(match)\
#                         else CONTRACTION_MAP.get(match.lower())
#         expandedContraction = firstChar+expandedContraction[1:]
#         return expandedContraction
    
#     reviewText = contractionsPattern.sub(expand_match, str(reviewText))
#     reviewText = re.sub("'", "", reviewText)
#     return reviewText

# df['reviewText'] = df['reviewText'].apply(expand_contractions)
# df.head(10)

# cont = Contractions('GoogleNews-vectors-negative300.bin')
expanded_words = []    
for word in df:
#     expanded_words.append(list(cont.expand_texts([word], precise=True))) 
    expanded_words.append(contractions.fix(word)) 
print(expanded_words[:200])

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> tokenizing words >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# ndf = df['reviewText']
tokens = word_tokenize(str(df))
print(tokens[:200])

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing non ascii >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

removedNonAscii = [w.encode("ascii", "ignore").decode() for w in tokens]
print(removedNonAscii[:100])

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> convert to lower case >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

lowerCase = [w.lower() for w in removedNonAscii]
print(lowerCase[:100])

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing punctuation >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

table = str.maketrans('', '', string.punctuation)
removedPunctuation = [w.translate(table) for w in lowerCase]
print(removedPunctuation[:100])

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> replacing numbers with string  >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# p = inflect.engine()
# stringNumbers = []
# for word in removedPunctuation:
#     if word.isdigit():
# #         new_word = num2word.to_card(15)
#         new_word = p.number_to_words(word)
#         stringNumbers.append(new_word)
#     else:
#         stringNumbers.append(word)
        
# # stringNumbers = [w.lower() for w in removedPunctuation]
# print(stringNumbers[:100])

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> removing stopwords >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

stop_words = set(stopwords.words('english'))
removedStopwords = [w for w in removedPunctuation if not w in stop_words]
print(removedStopwords[:100])

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> stemming words >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

stemmer = LancasterStemmer()
stems = []
for word in removedStopwords:
    stem = stemmer.stem(word)
    stems.append(stem)
    
print(stems[:100])

In [ ]:
# >>>>>>>>>>>>>>>>>>>>>>>>>>>>>> lemmatize_verbs >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

lemmatizer = WordNetLemmatizer()
lemmas = []
for word in stems:
    lemma = lemmatizer.lemmatize(word, pos='v')
    lemmas.append(lemma)
    
print(lemmas[:100])